## <font color=009DD2> <span style="background-color: #A9C4C0"> Imports

In [1]:
import sys,json
import glob, os
sys.path.append('/media/miplab-nas2/Data3/BMPD/hc_project/analysis/code/')

from seed_to_voxels_inprogress_nk import Seed2voxels

%matplotlib inline
%load_ext autoreload
%autoreload 2

## <font color=009DD2> <span style="background-color: #A9C4C0"> Initialization

### <font color=009DD2> I Configuration & parameters </font>

In [2]:
with open('../config/config_NK.json') as config_file:
    config = json.load(config_file) # load config file

# Parameters (maybe we should move them to config?)
signal='ai'
seed_names=['L-BA4'] # Define seed names (here, icap numbers)
target_name=['brain'] # Define structure to use as target (brain or spinal cord)


### <font color=009DD2> II Select files </font>

In [3]:
# One target per subject
data_target=[];ts_target_txt=[]
for subject_name in config['list_subjects']:
    data_target.append(config['main_dir'] + '1_nilearn-based/3_func_smoothed/sub-'+ subject_name + '/TA_results/MNI_srv4/TotalActivation/Activity_inducing.nii')
    ts_target_txt.append(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all/timeseries/sub-' + subject_name + '_' + target_name[0] + '_all_timeseries_ai')


ts_seed_txt = []
seed_name = 'L-BA4'
for subject_name in config['list_subjects']:
    ts_seed_txt.append(config['main_dir'] + config['seed2vox_dir'] +'/'+seed_name+'/timeseries/sub-' + subject_name + '_' + seed_name + '_timeseries_ai')
    if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + '/' +seed_name+'/'+target_name[0]+'_fc_maps/'):
        os.mkdir(config['main_dir'] + config['seed2vox_dir'] +'/'+seed_name+'/'+target_name[0]+'_fc_maps/')
    if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + '/' +seed_name+'/timeseries/'):
        os.mkdir(config['main_dir'] + config['seed2vox_dir'] +'/'+seed_name+'/timeseries/')

mask_target=config['main_dir'] + config['masks'][target_name[0].split('_')[0]]
 
# Create directories if needed
if not os.path.exists(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all/timeseries/'):
    os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all')
    os.mkdir(config['main_dir'] + config['seed2vox_dir'] + '/' +target_name[0]+'_all/timeseries/') # folder to store timeseries extraction

### <font color=009DD2> III Initialize class based on this </font>

In [4]:
seed2voxels=Seed2voxels(config,signal,seed_names,target_name) # initialize the function

## <font color=009DD2> <span style="background-color: #A9C4C0"> Data extraction


### <font color=009DD2> I. Time series extraction - Seed </font>

In [5]:
seed_timeseries=[]
seed_timeseries=seed2voxels.extract_data(img=data_target, mask='/media/miplab-nas2/Data3/BMPD/hc_project/seed_to_voxels/L-BA4/L-BA4.nii.gz', timeseries_txt=ts_seed_txt,
                                         run="extract",n_jobs=8) # Load iCAP timecourses as seed

Note: signals can only be extracted for targets for activity-inducing signals.


In [9]:
import numpy as np 
seed_timeseries_mean = []
for sub in range(len(seed2voxels.subject_names)):
    seed_timeseries_mean.append(np.mean(seed_timeseries[sub],axis=1))

### <font color=009DD2> II. Time series extraction - Target </font>

In [14]:
target_timeseries=seed2voxels.extract_data(img=data_target, mask=mask_target, timeseries_txt=ts_target_txt,
                run="extract",n_jobs=8) # Load voxelwise brain AI as target

Note: signals can only be extracted for targets for activity-inducing signals.


## <font color=009DD2> <span style="background-color: #A9C4C0"> Correlation analysis

In [16]:
output_img=[]
output_img=config['main_dir'] + config['seed2vox_dir'] + '/L-BA4/'+target_name[0]+'_fc_maps/' + str(len(config['list_subjects'])) + 'subjects_' + seed_name
seed2voxels.correlation_maps(seed_timeseries_mean,target_timeseries,mask=mask_target,output_img=output_img,Fisher=True,n_jobs=8)


## <font color=009DD2> <span style="background-color: #A9C4C0"> Statistics

### <font color=009DD2> Using non-parametric permutation testing on smoothed images</font>

In [ ]:
run_string = 'randomise -i ' + output_img['icap2'] + '_zcorr.nii -o ' + output_img['icap1'] + ' -1 -T -m ' + mask_target
os.system(run_string)